In [2]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [3]:
import os
os.chdir("../../")

In [4]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [7]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [ ]:
df_rumours.head()

In [8]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [9]:
x_data = obtain_path(
    df=df_rumours, 
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=20
)

x_data.shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:37<00:00, 148.35it/s]


(5568, 20, 384)

In [11]:
df_rumours.head(20)

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0
5,5.249425e+17,0,2014-10-22 15:16:52,"The University of Ottawa, just a five minute w...",1
6,5.249432e+17,0,2014-10-22 15:19:36,@ldobsonhughes @HuffPostCanada God Bless Heath...,1
7,5.249434e+17,0,2014-10-22 15:20:36,My Alma Mater. God. RT @ldobsonhughes: The Uni...,1
8,5.249441e+17,0,2014-10-22 15:23:16,@ldobsonhughes @InklessPW is 2 km from nasty p...,1
9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,2


In [10]:
x_data[4,:,:6]

array([[ 0.06482322,  0.00937483, -0.03336447, -0.04028158,  0.08319011,
         0.04379221],
       [ 0.01823091,  0.07951746,  0.03213233,  0.02682796,  0.00788967,
         0.04279753],
       [ 0.05382157,  0.00949206, -0.00965423, -0.03210687,  0.09826252,
         0.06938579],
       [-0.03186332,  0.05506539, -0.02402808, -0.02035064, -0.03055616,
        -0.03032097],
       [-0.02926038, -0.05390481, -0.03986266,  0.02331615,  0.13350497,
         0.00199842],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.

In [12]:
sbert_embeddings[4,:6]

tensor([-0.0293, -0.0539, -0.0399,  0.0233,  0.1335,  0.0020],
       dtype=torch.float64)

In [14]:
x_data[5,:,:6]

array([[ 0.06482322,  0.00937483, -0.03336447, -0.04028158,  0.08319011,
         0.04379221],
       [ 0.01823091,  0.07951746,  0.03213233,  0.02682796,  0.00788967,
         0.04279753],
       [ 0.05382157,  0.00949206, -0.00965423, -0.03210687,  0.09826252,
         0.06938579],
       [-0.03186332,  0.05506539, -0.02402808, -0.02035064, -0.03055616,
        -0.03032097],
       [-0.02926038, -0.05390481, -0.03986266,  0.02331615,  0.13350497,
         0.00199842],
       [ 0.03925574, -0.02373564, -0.01778829,  0.02274922,  0.11227943,
         0.12534747],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.

In [13]:
sbert_embeddings[5,:6]

tensor([-0.0553, -0.0195,  0.0161,  0.0007,  0.0900,  0.0530],
       dtype=torch.float64)

Baseline BiLSTM

In [18]:
num_epochs = 100
hidden_dim_sizes = [100]#[100, 200, 300]
num_layers = 1
bidirectional = True
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

history length = 20

In [19]:
size = 20
bilstm_history_20_kfold, best_bilstm_history_20_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=None,
    data_split_seed=123,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:34<00:00, 159.32it/s]





  0%|          | 0/1 [10:11<?, ?it/s]


KeyboardInterrupt: 